In [ ]:
import json
import logging
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, asdict
import os

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, SystemMessage

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Environment setup
OPENAI_API_KEY = "sk-abcdef1234567890abcdef1234567890abcdef12"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

@dataclass
class CareerPath:
    """Career path data structure"""
    name: str
    category: str
    description: str
    required_skills: List[str]
    academic_focus: List[str]
    salary_range: str
    growth_outlook: str
    personality_fit: List[str]
    related_interests: List[str]
    education_requirements: str

# Pydantic models for LangChain output parsing
class CareerRecommendation(BaseModel):
    """Single career recommendation model"""
    category: str = Field(description="The career category name")
    explanation: str = Field(description="Brief explanation connecting to user profile")
    confidence_score: float = Field(description="Confidence score from 0.0 to 1.0")

class RecommendationResponse(BaseModel):
    """Complete recommendation response model"""
    recommendations: List[CareerRecommendation] = Field(description="List of career recommendations")
    fallback_question: Optional[str] = Field(description="Question to ask if more info needed")
    reasoning: str = Field(description="Overall reasoning for recommendations")

class ClarificationResponse(BaseModel):
    """Response when clarification is needed"""
    recommendations: List[CareerRecommendation] = Field(default=[])
    fallback_question: str = Field(description="Specific question to ask user")
    reasoning: str = Field(description="Why clarification is needed")

class CareerDatabase:
    """Comprehensive career database with LangChain integration"""

    def __init__(self):
        self.careers = self._initialize_careers()
        self.categories = {
            "STEM": "Science, Technology, Engineering, Mathematics",
            "Arts & Creative": "Creative Arts, Design, Media Production",
            "Business & Finance": "Business, Finance, Marketing, Management",
            "Healthcare & Medicine": "Healthcare, Medicine, Wellness",
            "Education & Public Service": "Teaching, Government, Social Work",
            "Sports & Fitness": "Athletics, Fitness, Recreation"
        }

        # Initialize LangChain components
        self.llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0.7,
            max_tokens=800
        )

        # Set up memory for conversation context
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )

        # Initialize parsers
        self.recommendation_parser = PydanticOutputParser(pydantic_object=RecommendationResponse)
        self.clarification_parser = PydanticOutputParser(pydantic_object=ClarificationResponse)

        # Set up prompts
        self._setup_prompts()

    def _initialize_careers(self) -> List[CareerPath]:
        """Initialize comprehensive career database"""
        return [
            # STEM Careers
            CareerPath(
                name="Software Engineer",
                category="STEM",
                description="Design, develop, and maintain software applications and systems.",
                required_skills=["Programming", "Problem Solving", "Debugging", "Mathematics"],
                academic_focus=["Computer Science", "Mathematics", "Physics"],
                salary_range="$70,000 - $200,000+",
                growth_outlook="Excellent (22% growth)",
                personality_fit=["Analytical", "Creative", "Detail-oriented"],
                related_interests=["Technology", "Gaming", "Problem Solving", "Innovation"],
                education_requirements="Bachelor's in Computer Science"
            ),
            CareerPath(
                name="Data Scientist",
                category="STEM",
                description="Analyze complex data to extract insights for business decisions.",
                required_skills=["Statistics", "Programming", "Machine Learning", "Communication"],
                academic_focus=["Mathematics", "Statistics", "Computer Science"],
                salary_range="$80,000 - $180,000",
                growth_outlook="Excellent (31% growth)",
                personality_fit=["Analytical", "Curious", "Patient"],
                related_interests=["Mathematics", "Research", "Patterns", "Analysis"],
                education_requirements="Bachelor's/Master's in Data Science"
            ),
            CareerPath(
                name="Cybersecurity Specialist",
                category="STEM",
                description="Protect organizations from digital threats and security breaches.",
                required_skills=["Security Protocols", "Network Analysis", "Problem Solving", "Attention to Detail"],
                academic_focus=["Computer Science", "Information Security", "Mathematics"],
                salary_range="$75,000 - $160,000",
                growth_outlook="Excellent (35% growth)",
                personality_fit=["Analytical", "Vigilant", "Logical"],
                related_interests=["Technology", "Security", "Problem Solving", "Ethics"],
                education_requirements="Bachelor's in Cybersecurity or Computer Science"
            ),

            # Arts & Creative
            CareerPath(
                name="UX/UI Designer",
                category="Arts & Creative",
                description="Create user-friendly digital experiences and interfaces.",
                required_skills=["Design", "User Research", "Prototyping", "Creativity"],
                academic_focus=["Design", "Psychology", "Art", "Technology"],
                salary_range="$55,000 - $120,000",
                growth_outlook="Good (13% growth)",
                personality_fit=["Creative", "Empathetic", "Detail-oriented"],
                related_interests=["Design", "Technology", "Art", "User Psychology"],
                education_requirements="Bachelor's in Design or related field"
            ),
            CareerPath(
                name="Digital Marketing Specialist",
                category="Arts & Creative",
                description="Create and manage digital marketing campaigns across various platforms.",
                required_skills=["Content Creation", "Social Media", "Analytics", "Creativity"],
                academic_focus=["Marketing", "Communications", "Psychology", "Art"],
                salary_range="$45,000 - $85,000",
                growth_outlook="Good (10% growth)",
                personality_fit=["Creative", "Social", "Analytical"],
                related_interests=["Social Media", "Writing", "Design", "Trends"],
                education_requirements="Bachelor's in Marketing or Communications"
            ),

            # Business & Finance
            CareerPath(
                name="Marketing Manager",
                category="Business & Finance",
                description="Develop marketing strategies to promote products and services.",
                required_skills=["Communication", "Analytics", "Creativity", "Strategy"],
                academic_focus=["Business", "Psychology", "Communications"],
                salary_range="$50,000 - $150,000",
                growth_outlook="Good (10% growth)",
                personality_fit=["Outgoing", "Creative", "Strategic"],
                related_interests=["Psychology", "Social Media", "Communication", "Business"],
                education_requirements="Bachelor's in Marketing or Business"
            ),
            CareerPath(
                name="Financial Analyst",
                category="Business & Finance",
                description="Analyze financial data to guide investment and business decisions.",
                required_skills=["Financial Modeling", "Analytics", "Mathematics", "Communication"],
                academic_focus=["Finance", "Economics", "Mathematics", "Business"],
                salary_range="$60,000 - $120,000",
                growth_outlook="Good (6% growth)",
                personality_fit=["Analytical", "Detail-oriented", "Logical"],
                related_interests=["Mathematics", "Economics", "Investing", "Business"],
                education_requirements="Bachelor's in Finance or Economics"
            ),

            # Healthcare
            CareerPath(
                name="Registered Nurse",
                category="Healthcare & Medicine",
                description="Provide patient care in various healthcare settings.",
                required_skills=["Compassion", "Medical Knowledge", "Communication", "Critical Thinking"],
                academic_focus=["Biology", "Chemistry", "Health Sciences"],
                salary_range="$55,000 - $90,000",
                growth_outlook="Excellent (15% growth)",
                personality_fit=["Compassionate", "Reliable", "Caring"],
                related_interests=["Helping Others", "Healthcare", "Science", "Medicine"],
                education_requirements="Associate's or Bachelor's in Nursing"
            ),
            CareerPath(
                name="Physical Therapist",
                category="Healthcare & Medicine",
                description="Help patients recover from injuries and improve mobility.",
                required_skills=["Anatomy Knowledge", "Communication", "Patience", "Physical Strength"],
                academic_focus=["Biology", "Anatomy", "Kinesiology", "Health Sciences"],
                salary_range="$70,000 - $100,000",
                growth_outlook="Excellent (18% growth)",
                personality_fit=["Caring", "Patient", "Motivational"],
                related_interests=["Health", "Fitness", "Helping Others", "Science"],
                education_requirements="Doctoral degree in Physical Therapy"
            ),

            # Education
            CareerPath(
                name="High School Teacher",
                category="Education & Public Service",
                description="Educate and inspire students in specific subject areas.",
                required_skills=["Communication", "Patience", "Subject Expertise", "Leadership"],
                academic_focus=["Subject Specialization", "Education", "Psychology"],
                salary_range="$40,000 - $70,000",
                growth_outlook="Stable (4% growth)",
                personality_fit=["Patient", "Inspirational", "Organized"],
                related_interests=["Teaching", "Learning", "Working with Youth", "Education"],
                education_requirements="Bachelor's in Education + Teaching License"
            ),
            CareerPath(
                name="School Counselor",
                category="Education & Public Service",
                description="Provide academic and emotional support to students.",
                required_skills=["Counseling", "Communication", "Empathy", "Problem Solving"],
                academic_focus=["Psychology", "Education", "Counseling", "Social Work"],
                salary_range="$45,000 - $75,000",
                growth_outlook="Good (8% growth)",
                personality_fit=["Empathetic", "Patient", "Supportive"],
                related_interests=["Psychology", "Helping Others", "Education", "Mental Health"],
                education_requirements="Master's in School Counseling"
            ),

            # Sports & Fitness
            CareerPath(
                name="Sports Coach",
                category="Sports & Fitness",
                description="Train athletes and develop team strategies.",
                required_skills=["Leadership", "Communication", "Sports Knowledge", "Motivation"],
                academic_focus=["Physical Education", "Sports Science", "Psychology"],
                salary_range="$25,000 - $100,000+",
                growth_outlook="Good (11% growth)",
                personality_fit=["Motivational", "Energetic", "Competitive"],
                related_interests=["Sports", "Fitness", "Competition", "Teamwork"],
                education_requirements="Bachelor's in Sports Science"
            ),
            CareerPath(
                name="Personal Trainer",
                category="Sports & Fitness",
                description="Help individuals achieve their fitness and health goals.",
                required_skills=["Fitness Knowledge", "Motivation", "Communication", "Anatomy"],
                academic_focus=["Exercise Science", "Kinesiology", "Health", "Nutrition"],
                salary_range="$30,000 - $70,000",
                growth_outlook="Good (15% growth)",
                personality_fit=["Energetic", "Motivational", "Health-conscious"],
                related_interests=["Fitness", "Health", "Helping Others", "Sports"],
                education_requirements="Certification in Personal Training"
            )
        ]

    def _setup_prompts(self):
        """Set up LangChain prompt templates"""

        # Main recommendation prompt
        self.recommendation_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert career counselor AI. Analyze student profiles and provide personalized career recommendations.

Available Categories:
- STEM (Science, Technology, Engineering, Mathematics)
- Arts & Creative (Design, Media, Creative Arts)
- Business & Finance (Business, Marketing, Finance)
- Healthcare & Medicine (Medical, Wellness, Care)
- Education & Public Service (Teaching, Social Work)
- Sports & Fitness (Athletics, Coaching, Recreation)

Guidelines:
1. If the profile is too vague, ask ONE specific clarifying question
2. Provide 3 recommendations with confidence scores (0.0-1.0)
3. Connect recommendations directly to their interests and strengths
4. Include reasoning for your overall assessment

{format_instructions}"""),
            ("human", """Student Profile:
Interests: {interests}
Academic Strengths: {subjects}
Performance: {scores}

Please analyze this profile and provide career recommendations.""")
        ])

        # Career exploration prompt
        self.exploration_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a career exploration assistant. Help users understand career paths in depth.

Provide detailed insights about careers including:
- Day-to-day responsibilities
- Career progression paths
- Industry trends
- Skills development recommendations"""),
            ("human", "Tell me more about careers in {category}. Focus on {specific_interest}.")
        ])

        # Skills gap analysis prompt
        self.skills_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a skills development advisor. Analyze the gap between current student profile and career requirements.

Provide:
- Skills they already have
- Skills they need to develop
- Specific learning resources or steps
- Timeline recommendations"""),
            ("human", """Student wants to pursue: {target_career}
Current profile:
- Interests: {interests}
- Strengths: {subjects}
- Performance: {scores}

What skills do they need to develop?""")
        ])

    def get_career_recommendations(self, interests: str, subjects: str, scores: str) -> Dict:
        """Get career recommendations using LangChain"""
        try:
            # Create the chain
            chain = self.recommendation_prompt | self.llm | self.recommendation_parser

            # Add format instructions
            format_instructions = self.recommendation_parser.get_format_instructions()

            # Execute the chain
            result = chain.invoke({
                "interests": interests,
                "subjects": subjects,
                "scores": scores,
                "format_instructions": format_instructions
            })

            # Store conversation in memory
            self.memory.save_context(
                {"input": f"Interests: {interests}, Subjects: {subjects}, Performance: {scores}"},
                {"output": str(result)}
            )

            return result.dict()

        except Exception as e:
            logger.error(f"Error getting recommendations: {e}")
            return self._get_fallback_response(interests, subjects, scores)

    def _get_fallback_response(self, interests: str, subjects: str, scores: str) -> Dict:
        """Provide fallback response if LangChain fails"""
        # Simple rule-based fallback
        recommendations = []

        interests_lower = interests.lower()
        subjects_lower = subjects.lower()

        # Simple keyword matching
        if any(word in interests_lower for word in ['programming', 'coding', 'technology', 'computer']):
            recommendations.append({
                "category": "STEM",
                "explanation": "Your interest in technology aligns well with STEM careers",
                "confidence_score": 0.8
            })

        if any(word in interests_lower for word in ['art', 'design', 'creative', 'drawing']):
            recommendations.append({
                "category": "Arts & Creative",
                "explanation": "Your creative interests suggest a good fit for arts and design",
                "confidence_score": 0.7
            })

        if any(word in interests_lower for word in ['business', 'marketing', 'finance']):
            recommendations.append({
                "category": "Business & Finance",
                "explanation": "Your business interests align with this career path",
                "confidence_score": 0.7
            })

        # Ensure we have 3 recommendations
        while len(recommendations) < 3:
            remaining_categories = ["Healthcare & Medicine", "Education & Public Service", "Sports & Fitness"]
            for cat in remaining_categories:
                if not any(rec["category"] == cat for rec in recommendations):
                    recommendations.append({
                        "category": cat,
                        "explanation": "This could be a good alternative path to explore",
                        "confidence_score": 0.5
                    })
                    break

        return {
            "recommendations": recommendations[:3],
            "fallback_question": None,
            "reasoning": "Generated using fallback system due to processing error"
        }

    def explore_career_category(self, category: str, specific_interest: str = "general overview") -> str:
        """Explore careers in a specific category using LangChain"""
        try:
            chain = self.exploration_prompt | self.llm

            result = chain.invoke({
                "category": category,
                "specific_interest": specific_interest
            })

            return result.content

        except Exception as e:
            logger.error(f"Error exploring category: {e}")
            return f"Sorry, I couldn't retrieve detailed information about {category} careers at the moment."

    def analyze_skills_gap(self, target_career: str, interests: str, subjects: str, scores: str) -> str:
        """Analyze skills gap for a target career"""
        try:
            chain = self.skills_prompt | self.llm

            result = chain.invoke({
                "target_career": target_career,
                "interests": interests,
                "subjects": subjects,
                "scores": scores
            })

            return result.content

        except Exception as e:
            logger.error(f"Error analyzing skills gap: {e}")
            return "Sorry, I couldn't analyze the skills gap at the moment."

    def get_careers_by_category(self, category: str) -> List[CareerPath]:
        """Get all careers in a specific category"""
        return [career for career in self.careers if career.category == category]

    def get_conversation_history(self) -> str:
        """Get conversation history from memory"""
        return str(self.memory.buffer)

# Initialize career database with LangChain
career_db = CareerDatabase()

def display_career_recommendations(result: Dict):
    """Display career recommendations with enhanced formatting"""
    if not result:
        print("❌ Sorry, something went wrong. Please try again.")
        return

    if result.get("fallback_question"):
        print(f"🤔 **Need More Info:** {result['fallback_question']}")
        return

    if result.get("recommendations"):
        print("🎉 **Your AI-Powered Career Pathway Recommendations!**\n")

        # Show overall reasoning if available
        if result.get("reasoning"):
            print(f"**💭 Analysis:** {result['reasoning']}\n")

        for i, rec in enumerate(result["recommendations"], 1):
            category = rec.get('category', 'N/A')
            explanation = rec.get('explanation', 'No explanation provided.')
            confidence = rec.get('confidence_score', 0.0)

            # Display confidence as stars
            stars = "⭐" * int(confidence * 5)

            print(f"**{i}. {category}** {stars} ({confidence:.1f})")
            print(f"   {explanation}\n")

            # Show specific careers in this category
            category_key = category.split(' (')[0]  # Remove description
            specific_careers = career_db.get_careers_by_category(category_key)

            if specific_careers:
                print("   🎯 **Specific Career Options:**")
                for career in specific_careers[:2]:  # Show top 2
                    print(f"   • **{career.name}** - {career.description[:80]}...")
                    print(f"     💰 {career.salary_range} | 📈 {career.growth_outlook}")
                    print(f"     🎓 {career.education_requirements}")

            print("\n" + "="*60 + "\n")

def explore_careers_by_category():
    """Enhanced career exploration with LangChain insights"""
    print("🔍 **Career Categories Explorer (AI-Enhanced):**")
    categories = list(career_db.categories.keys())

    for i, (category, description) in enumerate(career_db.categories.items(), 1):
        print(f"{i}. {category} - {description}")

    try:
        choice = int(input("\nEnter category number to explore: ")) - 1
        if 0 <= choice < len(categories):
            selected_category = categories[choice]

            # Get AI insights about the category
            print(f"\n🤖 **AI Insights about {selected_category}:**")
            ai_insights = career_db.explore_career_category(selected_category)
            print(ai_insights)

            careers = career_db.get_careers_by_category(selected_category)

            print(f"\n🏢 **Specific Careers in {selected_category}:**\n")

            for career in careers:
                print(f"**{career.name}**")
                print(f"Description: {career.description}")
                print(f"💰 Salary: {career.salary_range}")
                print(f"📈 Growth: {career.growth_outlook}")
                print(f"🎓 Education: {career.education_requirements}")
                print(f"Skills: {', '.join(career.required_skills[:3])}")
                print(f"Best for: {', '.join(career.personality_fit[:2])}")
                print("-" * 50)
        else:
            print("Invalid choice.")
    except ValueError:
        print("Please enter a valid number.")

def skills_gap_analysis():
    """New feature: Skills gap analysis"""
    print("\n🎯 **Skills Gap Analysis**")
    print("-" * 30)

    target_career = input("What career are you interested in? ")
    interests = input("Your interests: ")
    subjects = input("Your strong subjects: ")
    scores = input("Your academic performance: ")

    print("\n🔄 Analyzing skills gap...")

    analysis = career_db.analyze_skills_gap(target_career, interests, subjects, scores)

    print("\n📊 **Skills Gap Analysis:**")
    print("=" * 40)
    print(analysis)

def main():
    """Enhanced main interactive function with LangChain features"""
    print("🎯 **AI-Powered Career Pathway Recommender with LangChain**")
    print("=" * 60)
    print("Discover career paths with advanced AI analysis!\n")

    while True:
        print("**Choose an option:**")
        print("1. Get AI-powered career recommendations")
        print("2. Explore careers by category (with AI insights)")
        print("3. Skills gap analysis")
        print("4. View conversation history")
        print("5. View career database stats")
        print("6. Exit")

        choice = input("\nEnter choice (1-6): ").strip()

        if choice == "1":
            print("\n🎯 **Tell Us About Yourself**")
            print("-" * 30)

            interests = input("\n**What are your interests and hobbies?**\n(e.g., coding, sports, art, music, reading...)\n> ")

            subjects = input("\n**What are your strongest/favorite subjects?**\n(e.g., Math, Science, English, History...)\n> ")

            scores = input("\n**How would you describe your academic performance?**\n(e.g., Excellent in Math, Good overall, Strong in Sciences...)\n> ")

            if not all([interests, subjects, scores]):
                print("⚠️ Please fill in all fields for best results.")
                continue

            print("\n🔄 AI is analyzing your profile...")
            result = career_db.get_career_recommendations(interests, subjects, scores)

            print("\n" + "="*60)
            display_career_recommendations(result)

        elif choice == "2":
            explore_careers_by_category()

        elif choice == "3":
            skills_gap_analysis()

        elif choice == "4":
            print("\n💬 **Conversation History:**")
            print("=" * 30)
            history = career_db.get_conversation_history()
            if history:
                print(history)
            else:
                print("No conversation history yet.")

        elif choice == "5":
            print(f"\n📊 **Database Stats:**")
            print(f"Total Careers: {len(career_db.careers)}")
            print(f"Categories: {len(career_db.categories)}")

            for category in career_db.categories:
                count = len(career_db.get_careers_by_category(category))
                print(f"  {category}: {count} careers")

        elif choice == "6":
            print("\n👋 Thank you for using the AI Career Recommender!")
            print("Keep exploring and following your passions! 🌟")
            break

        else:
            print("❌ Invalid choice. Please enter 1-6.")

def test_system():
    """Enhanced test system with LangChain features"""
    print("🧪 **Testing AI Career Recommendation System with LangChain**")
    print("=" * 60)

    test_cases = [
        {
            "name": "Tech Enthusiast",
            "interests": "Programming, artificial intelligence, robotics, gaming, problem solving",
            "subjects": "Mathematics, Computer Science, Physics",
            "scores": "Excellent in Math and Science, 95% average"
        },
        {
            "name": "Creative Artist",
            "interests": "Digital art, photography, music production, creative writing, social media",
            "subjects": "Art, English, Drama, Media Studies",
            "scores": "Outstanding in creative subjects, good overall"
        },
        {
            "name": "Future Helper",
            "interests": "Helping people, volunteering, psychology, medicine, community service",
            "subjects": "Biology, Psychology, Chemistry, English",
            "scores": "Strong in sciences and communication"
        }
    ]

    for test in test_cases:
        print(f"\n**Testing: {test['name']}**")
        print(f"Profile: {test['interests'][:50]}...")

        result = career_db.get_career_recommendations(
            test['interests'],
            test['subjects'],
            test['scores']
        )

        display_career_recommendations(result)

        # Test skills gap analysis
        if result.get('recommendations'):
            top_career = result['recommendations'][0]['category']
            print(f"\n🔍 **Skills Gap Analysis for {top_career}:**")

            gap_analysis = career_db.analyze_skills_gap(
                top_career,
                test['interests'],
                test['subjects'],
                test['scores']
            )
            print(gap_analysis[:200] + "..." if len(gap_analysis) > 200 else gap_analysis)

        print("\n" + "="*80 + "\n")

if __name__ == "__main__":
    print("🚀 **AI Career Recommender with LangChain**")
    print("Choose mode:")
    print("1. Interactive Mode")
    print("2. Test Mode")

    mode = input("Enter choice (1-2): ").strip()

    if mode == "1":
        main()
    elif mode == "2":
        test_system()
    else:
        print("Running interactive mode...")
        main()

🚀 **AI Career Recommender with LangChain**
Choose mode:
1. Interactive Mode
2. Test Mode
Enter choice (1-2): 1
🎯 **AI-Powered Career Pathway Recommender with LangChain**
Discover career paths with advanced AI analysis!

**Choose an option:**
1. Get AI-powered career recommendations
2. Explore careers by category (with AI insights)
3. Skills gap analysis
4. View conversation history
5. View career database stats
6. Exit

Enter choice (1-6): 1

🎯 **Tell Us About Yourself**
------------------------------

**What are your interests and hobbies?**
(e.g., coding, sports, art, music, reading...)
> coding

**What are your strongest/favorite subjects?**
(e.g., Math, Science, English, History...)
> english

**How would you describe your academic performance?**
(e.g., Excellent in Math, Good overall, Strong in Sciences...)
> Good overall


ERROR:__main__:Error getting recommendations: type object 'RecommendationResponse' has no attribute 'model_json_schema'



🔄 AI is analyzing your profile...

🎉 **Your AI-Powered Career Pathway Recommendations!**

**💭 Analysis:** Generated using fallback system due to processing error

**1. STEM** ⭐⭐⭐⭐ (0.8)
   Your interest in technology aligns well with STEM careers

   🎯 **Specific Career Options:**
   • **Software Engineer** - Design, develop, and maintain software applications and systems....
     💰 $70,000 - $200,000+ | 📈 Excellent (22% growth)
     🎓 Bachelor's in Computer Science
   • **Data Scientist** - Analyze complex data to extract insights for business decisions....
     💰 $80,000 - $180,000 | 📈 Excellent (31% growth)
     🎓 Bachelor's/Master's in Data Science


**2. Healthcare & Medicine** ⭐⭐ (0.5)
   This could be a good alternative path to explore

   🎯 **Specific Career Options:**
   • **Registered Nurse** - Provide patient care in various healthcare settings....
     💰 $55,000 - $90,000 | 📈 Excellent (15% growth)
     🎓 Associate's or Bachelor's in Nursing
   • **Physical Therapist** - H